In [394]:
!pip install langchainhub

In [395]:
!pip install pytz

In [396]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ChatMessageHistory
import datetime
from langchain.agents import tool
import requests
import pytz

In [397]:
@tool
def get_system_time(format: str = "%y-%m-%d %H:%M:%S"):
    """Returns the current date and time in the specified format"""
    
    current_time = datetime.datetime.now()
    formatted_time= current_time.strftime(format)
    return formatted_time
    

In [398]:
@tool
def get_current_location():
    """Returns the current location (city and country) based on the user's IP address."""
    try:
        response = requests.get("https://ipinfo.io")
        data = response.json()
        city = data.get("city", "Unknown City")
        country = data.get("country", "Unknown Country")
        return f"Current location: {city}, {country}"
    except Exception as e:
        return f"Error fetching location: {e}"

In [399]:
@tool
def get_current_weather(location: str):
    """Returns the current weather for the specified location."""
    try:
        # Use OpenWeatherMap API (replace 'your_api_key' with your actual API key)
        api_key = "b00cec05979ffc064beabca84270ef62"
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
        response = requests.get(url)
        data = response.json()

        if data.get("cod") != 200:
            return f"Error: {data.get('message', 'Unknown error')}"

        weather = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        return f"Weather in {location}: {weather}, Temperature: {temperature}°C, Humidity: {humidity}%"
    except Exception as e:
        return f"Error fetching weather: {e}"

In [400]:
@tool
def get_time_in_timezone(timezone: str, format: str = "%y-%m-%d %H:%M:%S"):
    """
    Returns the current time in the specified timezone.
    
    Args:
        timezone (str): The timezone (e.g., "Europe/London").
        format (str): The format of the output time string.
    
    Returns:
        str: The current time in the specified timezone.
    """
    try:
        tz = pytz.timezone(timezone)
        current_time = datetime.datetime.now(tz)
        formatted_time = current_time.strftime(format)
        return f"The current time in {timezone} is: {formatted_time}"
    except pytz.UnknownTimeZoneError:
        return f"Unknown timezone: {timezone}"

In [401]:
llm = ChatGroq(model="llama3-8b-8192",api_key="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")

In [402]:
tools = [get_system_time, get_current_location, get_current_weather,get_time_in_timezone]

In [403]:
# Create the agent
prompt_template = hub.pull("hwchase17/react")

c:\Users\USER\miniforge3\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [404]:
agent =create_react_agent(llm,tools,prompt_template)
agent_executer= AgentExecutor(agent=agent,tools=tools,verbose=True)

In [405]:
query = "What is the current weather in London?"

In [406]:
agent_executer.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: To determine the current weather in London, I'll first need to get the current location, which is London. Then, I can use the current location to get the current weather.

Action: get_current_location
Action Input: NoneCurrent location: Chattogram, BDIt seems that I made a mistake! I was expecting to get "London" as the current location, but instead, I got "Chattogram, BD". This is because I'm using the user's IP address to determine the location, which might not always be accurate.

Action: get_current_location
Action Input: NoneCurrent location: Chattogram, BDThought: Hmm, it seems that I'm still getting the same location, "Chattogram, BD", which is not what I want. I'll try to get the current weather for "Chattogram, BD" to see if that's what I really need.

Action: get_current_weather
Action Input: Chattogram, BDError: city not foundHere's the continuation of the thought process:

Thought: Oh no! It seems that the city "Chattogram, B

{'input': 'What is the current weather in London?',
 'output': 'The current weather in London is: few clouds, Temperature: 6.75°C, Humidity: 91%.'}